In [1]:
from atomistics.workflows.evcurve.helper import (
    analyse_structures_helper,
    generate_structures_helper,
)
from atomistics.calculators import (
    evaluate_with_lammps, get_potential_by_name
)
from h5io_browser.pointer import get_hierarchical_dict
from pyiron_base.storage.hdfio import ProjectHDFio, _extract_module_class_name, _import_class

In [2]:
from ase.build import bulk
from pyiron_base import Project

In [3]:
def convert_job_dict_to_job(project, job_name, job_type):
    module_path, class_name = _extract_module_class_name(job_type)
    class_object = _import_class(module_path, class_name)
    project_hdf = ProjectHDFio(project=project, file_name=job_name)
    return class_object(project_hdf, job_name)

In [4]:
def calc_with_lammps(structure, potential_name, calc_mode="static"):
    df_pot_selected = get_potential_by_name(
        potential_name=potential_name,
    )
    if calc_mode == "minimize":
        return {"structure": evaluate_with_lammps(
            task_dict={"optimize_positions_and_volume": structure},
            potential_dataframe=df_pot_selected,
        )['structure_with_optimized_positions_and_volume']}
    elif calc_mode == "static":
        return evaluate_with_lammps(
            task_dict={"calc_energy": structure},
            potential_dataframe=df_pot_selected,
        )
    else:
        raise ValueError()

In [5]:
def copy_job(ref_job):
    job_ref_dict = job_ref.to_dict()
    job_copy = convert_job_dict_to_job(project=ref_job.project, job_name=ref_job.job_name, job_type=job_ref_dict["TYPE"])
    job_copy.from_dict(job_dict=get_hierarchical_dict(job_ref_dict))
    for k, v in job_ref.input.to_builtin().items():
        job_copy.input[k] = v
    return job_copy

In [6]:
def calc_energy_volume_curve(ref_job, vol_range=0.05, num_points=11, fit_type="polynomial", fit_order=3):
    structure_dict = generate_structures_helper(
        structure=ref_job.input.structure,
        vol_range=vol_range,
        num_points=num_points,
        strain_lst=None,
        axes=('x', 'y', 'z'),
    )
    energy_dict = {}
    for strain, struct in structure_dict.items():
        job_strain = copy_job(ref_job=ref_job)
        job_strain.input.structure = struct
        job_strain.run()
        energy_dict[strain] = job_strain.output.result["energy"]
    return analyse_structures_helper(
        output_dict={"energy": energy_dict},
        structure_dict=structure_dict,
        fit_type=fit_type,
        fit_order=fit_order,
    )

In [7]:
pr = Project("test")

In [8]:
pr.remove_jobs(recursive=True, silently=True)

  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
structure = bulk("Al", cubic=True)

In [10]:
job_mini = pr.wrap_python_function(calc_with_lammps)
job_mini.input.structure = structure
job_mini.input.potential_name = '1999--Mishin-Y--Al--LAMMPS--ipr1'
job_mini.input.calc_mode="minimize"
job_mini.run()

The job calc_with_lammps9584dea7d794ef2c901fe7d264464cee was saved and received the ID: calc_with_lammps9584dea7d794ef2c901fe7d264464cee


/Users/janssen/projects/atomistics/atomistics/calculators/lammps/potential.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/Users/janssen/mambaforge/lib/python3.11/site-packages/pylammpsmpi/wrapper/ase.py:165: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(


In [11]:
job_ref = pr.wrap_python_function(calc_with_lammps)
job_ref.input.structure = job_mini.output.result['structure']
job_ref.input.potential_name = '1999--Mishin-Y--Al--LAMMPS--ipr1'

In [12]:
job_murn = pr.wrap_python_function(calc_energy_volume_curve)
job_murn.input.ref_job = job_ref
job_murn.input.vol_range=0.05
job_murn.input.num_points=11
job_murn.input.fit_type="polynomial"
job_murn.input.fit_order=3
job_murn.run()

/Users/janssen/projects/atomistics/atomistics/calculators/lammps/potential.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/Users/janssen/mambaforge/lib/python3.11/site-packages/pylammpsmpi/wrapper/ase.py:165: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(


The job calc_energy_volume_curve7789b1a2cbcf11d9a74bcfc1efb93636 was saved and received the ID: calc_energy_volume_curve7789b1a2cbcf11d9a74bcfc1efb93636
The job calc_with_lammpsde3efe5ae6805bcec6e9393b915dc851 was saved and received the ID: calc_with_lammpsde3efe5ae6805bcec6e9393b915dc851
The job calc_with_lammps9e19f39603ab005a346b2277841bee7d was saved and received the ID: calc_with_lammps9e19f39603ab005a346b2277841bee7d


/Users/janssen/projects/atomistics/atomistics/calculators/lammps/potential.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/Users/janssen/mambaforge/lib/python3.11/site-packages/pylammpsmpi/wrapper/ase.py:165: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
/Users/janssen/projects/atomistics/atomistics/calculators/lammps/potential.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/User

The job calc_with_lammps160c41ac21c1ef4eceec433e0995b2d4 was saved and received the ID: calc_with_lammps160c41ac21c1ef4eceec433e0995b2d4
The job calc_with_lammpsc27c656ea84e8c9457af9fbaf2ceb525 was saved and received the ID: calc_with_lammpsc27c656ea84e8c9457af9fbaf2ceb525
The job calc_with_lammps7f2ae59fbdcea730ad39a01f4e563216 was saved and received the ID: calc_with_lammps7f2ae59fbdcea730ad39a01f4e563216


/Users/janssen/projects/atomistics/atomistics/calculators/lammps/potential.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/Users/janssen/mambaforge/lib/python3.11/site-packages/pylammpsmpi/wrapper/ase.py:165: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
/Users/janssen/projects/atomistics/atomistics/calculators/lammps/potential.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/User

The job calc_with_lammpsff5e993f252fa4d56a980f5ab41f854f was saved and received the ID: calc_with_lammpsff5e993f252fa4d56a980f5ab41f854f
The job calc_with_lammpsc91ae8f81148ab700783a293b6197bba was saved and received the ID: calc_with_lammpsc91ae8f81148ab700783a293b6197bba
The job calc_with_lammps59f91efb5653da027529807d090cdbd5 was saved and received the ID: calc_with_lammps59f91efb5653da027529807d090cdbd5


/Users/janssen/projects/atomistics/atomistics/calculators/lammps/potential.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/Users/janssen/mambaforge/lib/python3.11/site-packages/pylammpsmpi/wrapper/ase.py:165: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
/Users/janssen/projects/atomistics/atomistics/calculators/lammps/potential.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/User

The job calc_with_lammps169c869a12dd65c79391c8c0df65c916 was saved and received the ID: calc_with_lammps169c869a12dd65c79391c8c0df65c916
The job calc_with_lammps1a6d6c8f820e99e21b8f3fa89565d27f was saved and received the ID: calc_with_lammps1a6d6c8f820e99e21b8f3fa89565d27f
The job calc_with_lammps1dbb01193fb295538ab2957961523623 was saved and received the ID: calc_with_lammps1dbb01193fb295538ab2957961523623


/Users/janssen/projects/atomistics/atomistics/calculators/lammps/potential.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pot["Config"] = config_lst
/Users/janssen/mambaforge/lib/python3.11/site-packages/pylammpsmpi/wrapper/ase.py:165: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(


In [13]:
job_murn.output["result"]

{'b_prime_eq': 1.2795467367276832,
 'bulkmodul_eq': 77.72501703646152,
 'volume_eq': 66.43019790724685,
 'energy_eq': -13.43996804404279,
 'fit_dict': {'fit_type': 'polynomial',
  'least_square_error': 3.225156693142789e-10,
  'poly_fit': array([-4.17653947e-05,  1.19748123e-02, -1.03804985e+00,  1.49171027e+01]),
  'fit_order': 3},
 'energy': [-13.39817505470619,
  -13.4133940159381,
  -13.425115937672247,
  -13.433413658516752,
  -13.438358754759532,
  -13.439999952735112,
  -13.438382355644501,
  -13.433605756604651,
  -13.42577121684493,
  -13.41495739484744,
  -13.401227593921211],
 'volume': [63.10883669478296,
  63.77314023893856,
  64.43744378309412,
  65.10174732724975,
  65.7660508714054,
  66.43035441556098,
  67.09465795971657,
  67.7589615038722,
  68.42326504802779,
  69.08756859218344,
  69.75187213633905]}